## Simulacija odziva procesa bez regulatora

![Blok shema motora](../images/motor_model.png)

In [8]:
%matplotlib widget
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
import matplotlib.pyplot as plt
import lib.simulator as sim

In [9]:
# model procesa
def motor(L_a, R_a, Kb, J, k_f, x0):
    def deriv(t, x, u):
        i_a = x[0]
        w = x[1]
        u_b = w*Kb
        di_a = 1/L_a*(u(0)-i_a*R_a-u_b)
        M_m = Kb*i_a
        dw = 1/J*(M_m+u(1)-k_f*w)
        return [di_a, dw]

    def out(t, x, u):
        return x[1]
    
    return (deriv, out, x0) 

In [10]:
# model cjelokupnog sustava

blocks = [
    sim.step(y0=0, y1=1, t_step=1),  #0                  ulazni signal - napon motora
    sim.step(y0=0, y1=-0.5, t_step=15), #1                  ulazni signal - vanjski moment
    motor(L_a=0.3, R_a=3, Kb=1, J=0.5, k_f=0.1, x0=[0.0, 0.0]),   # parametri motora
    ]

connections = [
    ((0, 0), (2, 0)),
    ((1, 0), (2, 1)),
]

outputs = [0, 1, 2]

t, y = sim.simulate(blocks, connections, outputs, t_final=30.0)

In [11]:
# vremenski odziv
fig = plt.figure()
fig.set_label('Odziv procesa')
ax1=plt.subplot(2, 1, 1)
plt.plot(t, y[:,2])
plt.grid()
plt.legend(['Brzina vrtnje'])
ax2=plt.subplot(2, 1, 2)
plt.plot(t, y[:,0], t, y[:,1])
plt.grid()
plt.legend(['Ulaz (napon)', 'Vanjski moment'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …